In [ ]:
#!pip install pgmpy

In [64]:
# 使用するパッケージ（ライブラリと関数）を定義
#独立性の検定に使用
from pgmpy.estimators import ConstraintBasedEstimator
import pandas as pd

In [65]:
df = pd.read_csv("test_data.csv",encoding = "cp932")

In [63]:
import pandas as pd
import numpy as np
from pgmpy.estimators import ConstraintBasedEstimator
data = pd.DataFrame(np.random.randint(0, 2, size=(5000, 5)), columns=list('ABCDE'))
data['F'] = data['A'] + data['B'] + data ['C']
est = ConstraintBasedEstimator(data)
skel, sep_sets = est.estimate_skeleton()
skel.edges()
# all independencies are unconditional:
sep_sets
data = pd.DataFrame(np.random.randint(0, 2, size=(5000, 3)), columns=list('XYZ'))
data['X'] += data['Z']
data['Y'] += data['Z']
est = ConstraintBasedEstimator(data)
skel, sep_sets = est.estimate_skeleton()
skel.edges()
# X, Y dependent, but conditionally independent given Z:
sep_sets

TypeError: 'module' object is not callable

In [52]:
est = ConstraintBasedEstimator.ConstraintBasedEstimator(df)

In [57]:
skel, seperating_sets = est.estimate_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

AttributeError: 'ConstraintBasedEstimator' object has no attribute 'test_conditional_independence'

In [55]:
pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = est.pdag_to_dag(pdag)
print("DAG edges:        ", model.edges())

NameError: name 'skel' is not defined

In [42]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

# ベイジアンネットワークの構造
model = BayesianModel([
    # ('親ノード', '子ノード')
    ('battery', 'gauge'),
    ('fuel', 'gauge'),
])

battery_cpd = TabularCPD(
    variable='battery',
    variable_card=2,
    # values[0]: empty, values[1]: full
    values=[[.1], [.9]])

fuel_cpd = TabularCPD(
    variable='fuel',
    variable_card=2,
    values=[[.1], [.9]])

gauge_cpd = TabularCPD(
    variable='gauge',
    variable_card=2,
    values = [[.9, .8, .8, .2],
             [.1, .2, .2, .8]],
    evidence = ['battery', 'fuel'],
    evidence_card=[2, 2]
)

model.add_cpds(battery_cpd, fuel_cpd, gauge_cpd)

In [43]:
from pgmpy.inference import VariableElimination

model_infer = VariableElimination(model)
fuel0_prob_given_gauge0 = model_infer.query(variables=['fuel'], evidence={'gauge': 0})
print(fuel0_prob_given_gauge0.values[0])

Eliminating: battery: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.33it/s]

0.2571428571428572


In [44]:
import pandas as pd
df = pd.DataFrame()
df['t'] = [1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2] #3分類
df['a'] = [2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 0, 0, 0, 1, 1, 2, 2, 2] #3分類
df['h'] = [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1] #2分類

In [45]:
from pgmpy.models import BayesianModel
model = BayesianModel([('t','h'),('a','h')])

In [46]:
model.fit(df) #条件は省略。標準ではオーバーフィットに特に注意

print(model.get_cpds('t'))
print(model.get_cpds('a'))
print(model.get_cpds('h'))

Finding Elimination Order: : 100%|██████████████████████████████████████████████████████| 1/1 [06:37<00:00, 397.83s/it]

+------+-----+
| t(0) | 0.2 |
+------+-----+
| t(1) | 0.5 |
+------+-----+
| t(2) | 0.3 |
+------+-----+
+------+------+
| a(0) | 0.15 |
+------+------+
| a(1) | 0.4  |
+------+------+
| a(2) | 0.45 |
+------+------+
+------+------+--------------------+------+------+------+------+------+------+------+
| a    | a(0) | a(0)               | a(0) | a(1) | a(1) | a(1) | a(2) | a(2) | a(2) |
+------+------+--------------------+------+------+------+------+------+------+------+
| t    | t(0) | t(1)               | t(2) | t(0) | t(1) | t(2) | t(0) | t(1) | t(2) |
+------+------+--------------------+------+------+------+------+------+------+------+
| h(0) | 0.5  | 0.3333333333333333 | 0.5  | 1.0  | 0.0  | 0.0  | 1.0  | 0.6  | 0.0  |
+------+------+--------------------+------+------+------+------+------+------+------+
| h(1) | 0.5  | 0.6666666666666666 | 0.5  | 0.0  | 1.0  | 1.0  | 0.0  | 0.4  | 1.0  |
+------+------+--------------------+------+------+------+------+------+------+------+


In [47]:
from pgmpy.inference import VariableElimination
ve = VariableElimination(model)

#t=1,h=1とした場合のaは？
print(ve.map_query(variables=['a'], evidence={'t':1, 'h':1}))

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

{'a': 1}


In [48]:
#t=0,1,2とした場合のa,hは？
for i in [0,1,2]:
    print(ve.query(variables=['a', 'h'], evidence={'t':i}))


0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]

0it [00:00, ?it/s]


0it [00:00, ?it/s]

Finding Elimination Order: : : 0it [00:00, ?it/s]


0it [00:00, ?it/s]



0it [00:00, ?it/s]


Finding Elimination Order: : : 0it [00:00, ?it/s]



0it [00:00, ?it/s]

+------+------+------------+
| h    | a    |   phi(h,a) |
+======+======+============+
| h(0) | a(0) |     0.0750 |
+------+------+------------+
| h(0) | a(1) |     0.4000 |
+------+------+------------+
| h(0) | a(2) |     0.4500 |
+------+------+------------+
| h(1) | a(0) |     0.0750 |
+------+------+------------+
| h(1) | a(1) |     0.0000 |
+------+------+------------+
| h(1) | a(2) |     0.0000 |
+------+------+------------+
+------+------+------------+
| a    | h    |   phi(a,h) |
+======+======+============+
| a(0) | h(0) |     0.0500 |
+------+------+------------+
| a(0) | h(1) |     0.1000 |
+------+------+------------+
| a(1) | h(0) |     0.0000 |
+------+------+------------+
| a(1) | h(1) |     0.4000 |
+------+------+------------+
| a(2) | h(0) |     0.2700 |
+------+------+------------+
| a(2) | h(1) |     0.1800 |
+------+------+------------+
+------+------+------------+
| h    | a    |   phi(h,a) |
+======+======+============+
| h(0) | a(0) |     0.0750 |
+------+------